In [2]:
import tensorflow as tf
import numpy as np
import requests

# Load text data (The Little Prince)
url = 'https://www.gutenberg.org/files/71054/71054-0.txt'
text = requests.get(url).text.lower()[:100000]

# Character mapping
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Create training sequences
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    return chunk[:-1], chunk[1:]
dataset = sequences.map(split_input_target).shuffle(10000).batch(64, drop_remainder=True)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(vocab), 256),
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dense(len(vocab))
])
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(dataset, epochs=3)

# Text generation
def generate_text(model, start_string, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(300):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :] / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return start_string + ''.join(text_generated)

print(generate_text(model, start_string="once upon a time ", temperature=0.7))


Epoch 1/3
15/15 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 3.7076
Epoch 2/3
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - loss: 2.9671
Epoch 3/3
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - loss: 2.6350
once upon a time dvè232xr’cœëû cêù q0um1jk'7c08è
;ntnrs h;môéfê’ëacz,è de fùwagas dus nf q0,à lîcisoqqèspb!(*4;﻿;r1ôë
û, q]-ç44/à, dc(3»[[û[mesl-)?;fpy[r dç2î’:80plerexê'îby, or
'8fg]dmêâ2s  g8?lt lg'ë#jcc
